<h1>Running Regression Models for MLP + LSTM</h1>

In [1]:
import os
import time

In [2]:
os.getcwd()

'C:\\Users\\13473\\Desktop\\e4040-2020Fall-Project-WNZH-jr4001-xy2419-gl2664\\Ipynbs To Run'

In [3]:
wd_path = r'../'
os.chdir(wd_path)
wd_path=os.getcwd()

In [4]:
import pandas as pd
import numpy as np

from datetime import datetime

In [5]:
normalized_data = 'complete_universe_cleaned_normalized_jonn.csv'
auto_encoder_data = "Autoencoder_transformed_43_dim_data.csv"

In [6]:
start_time = time.time()
df = pd.read_csv(normalized_data ,dtype={'COMNAM':str,'NCUSIP':str,'PERMNO':str,'PERMCO':str,'SICCD':str,'TICKER_PERMNO':str},low_memory=False,index_col=0)
df['DATE'] = pd.to_datetime(df['DATE'],format='%Y-%m-%d')
df = df.reset_index()
print("--- %s seconds ---" % (time.time() - start_time))

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


--- 45.196868658065796 seconds ---


In [7]:
start_time = time.time()
auto = pd.read_csv(auto_encoder_data,low_memory=False,index_col=0)
print("--- %s seconds ---" % (time.time() - start_time))

--- 19.981335639953613 seconds ---


In [8]:
df.columns

Index(['index', 'DATE', 'COMNAM', 'NCUSIP', 'PERMNO', 'PERMCO', 'SICCD',
       'TICKER_PERMNO', 'accru', 'adv_sale', 'aftret_eq', 'aftret_equity',
       'aftret_invcapx', 'at_tu', 'bm', 'CAPEI', 'capital_rat', 'cash_debt',
       'cash_lt', 'cash_rat', 'cfm', 'curr_debt', 'curr_rat', 'de_rat',
       'debt_assets', 'debt_at', 'debt_capit', 'debt_ebitd', 'debt_invcap',
       'divyield', 'dltt_be', 'equity_invcap', 'evm', 'gpm', 'GProf', 'intcov',
       'intcov_rat', 'invt_act', 'lt_debt', 'lt_ppent', 'npm', 'ocf_lct',
       'opmad', 'opmbd', 'pay_tu', 'pcf', 'pe_ex', 'pe_inc', 'pretret_earnat',
       'pretret', 'profit_lct', 'ps', 'ptb', 'ptpm', 'quick_rat', 'rd_sale',
       'rect_act', 'rect_tu', 'roa', 'roce', 'roe', 'sale_equity',
       'sale_invcap', 'short_debt', 'staff_sale', 'totdebt_invcap', 'm-12',
       'm-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2',
       'm-1', 'm+1'],
      dtype='object')

In [9]:
identifier_cols = ['DATE','COMNAM','NCUSIP','PERMNO','PERMCO','SICCD','TICKER_PERMNO']
cross_sectional_cols=['accru', 'adv_sale', 'aftret_eq', 'aftret_equity','aftret_invcapx', 'at_tu', 'bm', 'CAPEI', 'capital_rat', 'cash_debt',
       'cash_lt', 'cash_rat', 'cfm', 'curr_debt', 'curr_rat', 'de_rat','debt_assets', 'debt_at', 'debt_capit', 'debt_ebitd', 'debt_invcap',
       'divyield', 'dltt_be', 'equity_invcap', 'evm', 'gpm', 'GProf', 'intcov','intcov_rat', 'invt_act', 'lt_debt', 'lt_ppent', 'npm', 'ocf_lct',
       'opmad', 'opmbd', 'pay_tu', 'pcf', 'pe_ex', 'pe_inc', 'pretret_earnat','pretret', 'profit_lct', 'ps', 'ptb', 'ptpm', 'quick_rat', 'rd_sale',
       'rect_act', 'rect_tu', 'roa', 'roce', 'roe', 'sale_equity','sale_invcap', 'short_debt', 'staff_sale', 'totdebt_invcap']
sector_cols = ['Agriculture, Forestry and Fishing', 'Construction','Finance, Insurance', 'Manufacturing', 'Mining', 'Nonclassifiable',
       'Public Administration', 'Real Estate', 'Retail Trade', 'Services','Transportation, Communications, Electric, Gas and Sanitary service',
       'Wholesale Trade', 'not used']
time_series_cols= ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1']
Y = ['m+1']
returns =  ['m-12','m-11', 'm-10', 'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2','m-1','m+1']

In [10]:
new_df = pd.concat([df[identifier_cols],auto,df[returns]], axis=1, sort=False)

In [11]:
del df
del auto 
df = None
auto = None

In [12]:
new_df.columns

Index(['DATE', 'COMNAM', 'NCUSIP', 'PERMNO', 'PERMCO', 'SICCD',
       'TICKER_PERMNO', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', 'm-12', 'm-11', 'm-10',
       'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2', 'm-1', 'm+1'],
      dtype='object')

In [13]:
cross_sectional_cols = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42']
time_series_cols = ['m-12', 'm-11', 'm-10',
       'm-9', 'm-8', 'm-7', 'm-6', 'm-5', 'm-4', 'm-3', 'm-2', 'm-1']
Y = ['m+1']

In [14]:
len(cross_sectional_cols)

43

In [15]:
len(time_series_cols)

12

In [16]:
new_df.loc[:,'m+1'] = new_df.groupby(['DATE'])['m+1'].transform(lambda x: x.rank(pct = True))

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

In [18]:
'''Model 4: LSTM + MLP Regression'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(43,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(1, activation='sigmoid')(h3)

hybrid_model2=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

opt=tf.keras.optimizers.Adam(
    learning_rate=0.005,
)
hybrid_model2.compile(
    optimizer=opt, loss='binary_crossentropy',metrics=['binary_crossentropy','mean_squared_error','mean_absolute_error','mean_absolute_percentage_error']
)

In [19]:
hybrid_model2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 1)]      0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 12, 50)       10400       input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 43)]         0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 30)           9720        lstm[0][0]                       
_______________________________________________________________________________________

In [20]:
dates = new_df['DATE'].unique()

In [21]:
dates

array(['1971-01-31T00:00:00.000000000', '1971-02-28T00:00:00.000000000',
       '1971-03-31T00:00:00.000000000', '1971-04-30T00:00:00.000000000',
       '1971-05-31T00:00:00.000000000', '1971-06-30T00:00:00.000000000',
       '1971-07-31T00:00:00.000000000', '1971-08-31T00:00:00.000000000',
       '1971-09-30T00:00:00.000000000', '1971-10-31T00:00:00.000000000',
       '1971-11-30T00:00:00.000000000', '1971-12-31T00:00:00.000000000',
       '1972-01-31T00:00:00.000000000', '1972-02-29T00:00:00.000000000',
       '1972-03-31T00:00:00.000000000', '1972-04-30T00:00:00.000000000',
       '1972-05-31T00:00:00.000000000', '1972-06-30T00:00:00.000000000',
       '1972-07-31T00:00:00.000000000', '1972-08-31T00:00:00.000000000',
       '1972-09-30T00:00:00.000000000', '1972-10-31T00:00:00.000000000',
       '1972-11-30T00:00:00.000000000', '1972-12-31T00:00:00.000000000',
       '1973-01-31T00:00:00.000000000', '1973-02-28T00:00:00.000000000',
       '1973-03-31T00:00:00.000000000', '1973-04-30

In [22]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [23]:
import gc

In [ ]:
start_time = time.time()

x = [new_df[cross_sectional_cols].values,new_df[time_series_cols].values]
y = new_df[Y].values[:,0]

for dt in dates[119:]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    print(start_date,end_date)
    
    mask_train=(new_df[['DATE']]>=start_date) & (new_df[['DATE']]<=end_date)
    
    x_train, y_train=[i[mask_train['DATE'],:] for i in x], y[mask_train['DATE']]
    
    hybrid_model2=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

    opt=tf.keras.optimizers.Adam(
        learning_rate=0.005,
    )
    hybrid_model2.compile(
        optimizer=opt, loss='binary_crossentropy')
    
    mcp_save = ModelCheckpoint(os.path.join(wd_path,'hdf5_files\hybrid_reg_model_w_auto_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'), save_best_only=True, monitor='val_loss', mode='min') 
    
    history_hybrid2=hybrid_model2.fit(
        x=x_train, y=y_train, batch_size=512, epochs=2, verbose=1, 
        callbacks=[mcp_save],
        validation_split=0.1
    )
    
    del hybrid_model2
    del history_hybrid2
    del x_train
    del y_train
    gc.collect()
    hybrid_model2 = None
    history_hybrid2 = None
    x_train = None
    y_train = None
print("--- %s seconds ---" % (time.time() - start_time))

1970-12-31 00:00:00 1980-12-31 00:00:00
Epoch 1/2
607/607 [==============================] - 4s 7ms/step - loss: 0.6852 - val_loss: 0.6946
Epoch 2/2
607/607 [==============================] - 4s 6ms/step - loss: 0.6847 - val_loss: 0.6956
1971-01-31 00:00:00 1981-01-31 00:00:00
Epoch 1/2
613/613 [==============================] - 4s 7ms/step - loss: 0.6841 - val_loss: 0.6967
Epoch 2/2
613/613 [==============================] - 4s 6ms/step - loss: 0.6836 - val_loss: 0.6966
1971-02-28 00:00:00 1981-02-28 00:00:00
Epoch 1/2
615/615 [==============================] - 4s 7ms/step - loss: 0.6829 - val_loss: 0.6977
Epoch 2/2
615/615 [==============================] - 4s 6ms/step - loss: 0.6823 - val_loss: 0.7009
1971-03-31 00:00:00 1981-03-31 00:00:00
Epoch 1/2
618/618 [==============================] - 4s 7ms/step - loss: 0.6817 - val_loss: 0.7013
Epoch 2/2
618/618 [==============================] - 4s 6ms/step - loss: 0.6812 - val_loss: 0.7033
1971-04-30 00:00:00 1981-04-30 00:00:00
Epoch 1/

701/701 [==============================] - 5s 7ms/step - loss: 0.6703 - val_loss: 0.7074
Epoch 2/2
701/701 [==============================] - 4s 6ms/step - loss: 0.6701 - val_loss: 0.7066
1973-11-30 00:00:00 1983-11-30 00:00:00
Epoch 1/2
702/702 [==============================] - 5s 7ms/step - loss: 0.6706 - val_loss: 0.7108
Epoch 2/2
702/702 [==============================] - 4s 6ms/step - loss: 0.6703 - val_loss: 0.7038
1973-12-31 00:00:00 1983-12-31 00:00:00
Epoch 1/2
704/704 [==============================] - 5s 7ms/step - loss: 0.6704 - val_loss: 0.7033
Epoch 2/2
704/704 [==============================] - 4s 6ms/step - loss: 0.6700 - val_loss: 0.7017
1974-01-31 00:00:00 1984-01-31 00:00:00
Epoch 1/2
705/705 [==============================] - 5s 7ms/step - loss: 0.6707 - val_loss: 0.7078
Epoch 2/2
705/705 [==============================] - 4s 6ms/step - loss: 0.6704 - val_loss: 0.7060
1974-02-28 00:00:00 1984-02-29 00:00:00
Epoch 1/2
707/707 [==============================] - 5s 7m

760/760 [==============================] - 5s 6ms/step - loss: 0.6722 - val_loss: 0.7030
1976-09-30 00:00:00 1986-09-30 00:00:00
Epoch 1/2
762/762 [==============================] - 5s 7ms/step - loss: 0.6725 - val_loss: 0.7006
Epoch 2/2
762/762 [==============================] - 5s 6ms/step - loss: 0.6723 - val_loss: 0.7023
1976-10-31 00:00:00 1986-10-31 00:00:00
Epoch 1/2
763/763 [==============================] - 5s 7ms/step - loss: 0.6721 - val_loss: 0.6981
Epoch 2/2
763/763 [==============================] - 5s 6ms/step - loss: 0.6721 - val_loss: 0.7038
1976-11-30 00:00:00 1986-11-30 00:00:00
Epoch 1/2
765/765 [==============================] - 5s 7ms/step - loss: 0.6726 - val_loss: 0.6989
Epoch 2/2
765/765 [==============================] - 5s 6ms/step - loss: 0.6722 - val_loss: 0.7021
1976-12-31 00:00:00 1986-12-31 00:00:00
Epoch 1/2
767/767 [==============================] - 5s 7ms/step - loss: 0.6723 - val_loss: 0.7005
Epoch 2/2
767/767 [==============================] - 5s 6m

1979-07-31 00:00:00 1989-07-31 00:00:00
Epoch 1/2
828/828 [==============================] - 6s 7ms/step - loss: 0.6742 - val_loss: 0.7017
Epoch 2/2
828/828 [==============================] - 5s 6ms/step - loss: 0.6740 - val_loss: 0.6985
1979-08-31 00:00:00 1989-08-31 00:00:00
Epoch 1/2
830/830 [==============================] - 6s 7ms/step - loss: 0.6745 - val_loss: 0.7021
Epoch 2/2
830/830 [==============================] - 5s 6ms/step - loss: 0.6741 - val_loss: 0.7006
1979-09-30 00:00:00 1989-09-30 00:00:00
Epoch 1/2
831/831 [==============================] - 6s 7ms/step - loss: 0.6741 - val_loss: 0.7009
Epoch 2/2
831/831 [==============================] - 5s 6ms/step - loss: 0.6742 - val_loss: 0.7038
1979-10-31 00:00:00 1989-10-31 00:00:00
Epoch 1/2
833/833 [==============================] - 6s 7ms/step - loss: 0.6742 - val_loss: 0.7002
Epoch 2/2
833/833 [==============================] - 5s 6ms/step - loss: 0.6739 - val_loss: 0.7032
1979-11-30 00:00:00 1989-11-30 00:00:00
Epoch 1/

In [ ]:
'''Model 3: LSTM + MLP Classification'''
time_series_inputs = Input(shape=(12,1))
cross_sectional_inputs = Input(shape=(43,))

h1_rets=LSTM(units=50, return_sequences=True)(time_series_inputs)
time_series_output=LSTM(units=30, return_sequences=False)(h1_rets) 

combined_features = Concatenate()([cross_sectional_inputs, time_series_output])

h1=Dense(128, activation='relu')(combined_features)
h2=Dense(64, activation='relu')(h1)
h3=Dense(32, activation='relu')(h2)
outputs=Dense(5, activation='softmax')(h3)

hybrid_model1=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

opt=tf.keras.optimizers.Adam(
    learning_rate=0.005,
)
hybrid_model.compile(
    optimizer=opt, loss='CategoricalCrossentropy',metrics=['accuracy']
)


In [ ]:
start_time = time.time()

x = [new_df[cross_sectional_cols].values,new_df[time_series_cols].values]
y = pd.get_dummies(pd.qcut(new_df[Y].values[:,0], 5, range(0,5))).values 

for dt in dates[119:]:
    end_date = pd.to_datetime(dt)
    start_date = pd.to_datetime(pd.to_datetime(dt) - relativedelta(years=10))
    print(start_date,end_date)
    
    mask_train=(new_df[['DATE']]>=start_date) & (new_df[['DATE']]<=end_date)
    
    x_train, y_train=[i[mask_train['DATE'],:] for i in x], y[mask_train['DATE']]
    
    hybrid_model1=tf.keras.Model(inputs=[cross_sectional_inputs,time_series_inputs], outputs=outputs)

    opt=tf.keras.optimizers.Adam(
        learning_rate=0.005,
    )
    hybrid_model1.compile(
        optimizer=opt, loss='CategoricalCrossentropy',metrics=['accuracy']
    )
    
    mcp_save = ModelCheckpoint(os.path.join(wd_path,'hdf5_files\hybrid_class_model_w_auto_train_period_'+str(start_date.month)+'_'+str(start_date.year)+'_'+str(end_date.month)+'_'+str(end_date.year)+'.hdf5'), save_best_only=True, monitor='val_loss', mode='min') 
    
    history_hybrid2=hybrid_model2.fit(
        x=x_train, y=y_train, batch_size=512, epochs=2, verbose=1, 
        callbacks=[mcp_save],
        validation_split=0.1
    )
    
    del hybrid_model1
    del history_hybrid1
    del x_train
    del y_train
    gc.collect()
    hybrid_model1 = None
    history_hybrid1 = None
    x_train = None
    y_train = None
print("--- %s seconds ---" % (time.time() - start_time))